## 스트림 처리의 기초
- DStream: 자바나 파이썬 객체에 대한 저수준 연산만 지원하기 때문에 고수준 최적화 기법을 활용하는데 한계가 있음. 이벤트 시간 처리를 지원하지 않음
- 구조적 Streaming: 2016년 DataFrame 기반으로 구현된 신규 스트리밍 API. DataFrame이나 Dataset과 통합 가능. 이벤트 시간 데이터 처리를 지원
- 스트리밍의 장점: 사용자 응답을 위한 대기 시간이 짧으며, 연산 결과의 증분이 생성되므로 효율적임
- 스트리밍의 과제: 높은 데이터 처리량, 빠른 처리, 순서가 뒤섞인 데이터 처리, 입력값과 출력값에 대한 트랜잭션 보장 등

### Stream processing
- 신규 데이터를 끊임없이 처리해 결과를 만드는 행위. 입력 데이터 (이벤트 스트림)이 도착한 후 다양한 쿼리 연산을 수행하여 결과를 출력하거나 외부 sink 시스템에 데이터를 저장함
- 배치 처리: 고정된 입력 데이터셋을 다루는 데이터 처리. 결과가 한번 산출됨. 스트리밍과 배치 연산은 조인하여 사용되는 경우가 있음

### Stream 처리 사례
1. Notification and Alerting: 특정 이벤트나 이벤트 패턴 탐지시 이를 알림
2. 실시간 리포트: 실시간 대시보드 구현
3. 실시간 데이터 제공: 예시는 웹 UI에서 최신 방문자 수 조회 (구글 애널리틱스)
4. 실시간 의사결정: 신규 입력을 분석하고 비즈니스 로직에 따라 처리함. 예시는 부정행위에 속하는 카드 트랜잭션 거부
5. 온라인 머신러닝: 실시간 데이터와 이력 데이터를 조합해서 모델 학습

## 구조적 스트리밍의 기초 
- 스파크 SQL 엔진 기반의 스트림 처리 프레임워크
- 스트림 데이터를 데이터가 계속 추가되는 테이블처럼 다룸. 쿼리 구문을 변경하지 않고 계속 사용
- 출력 모드: append (싱크에 신규 레코드 추가), update (대상 레코드 갱신), complete (전체 출력 내용 재작성)
- 트리거: 데이터 출력 시점을 정의
- 몇가지 제약을 제외하고 기본적으로 정적인 구조적 API의 모든 트랜스포메이션을 스트리밍 DataFrame에서도 사용 가능

### 구조적 스트리밍 활용
- 데이터셋: 인간 행동 인지를 위한 이기종 (heterogeneity) 데이터셋. 사용자의 자전거 타기, 앉기, 일어서기, 걷기 등의 활동을 기록한 센서 데이터

In [1]:
val static = spark.read.json("c:/SparkDG/data/activity-data/")
val dataSchema = static.schema

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.123.105:4041
SparkContext available as 'sc' (version = 2.3.2, master = local[*], app id = local-1597802677585)
SparkSession available as 'spark'


static: org.apache.spark.sql.DataFrame = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(Arrival_Time,LongType,true), StructField(Creation_Time,LongType,true), StructField(Device,StringType,true), StructField(Index,LongType,true), StructField(Model,StringType,true), StructField(User,StringType,true), StructField(gt,StringType,true), StructField(x,DoubleType,true), StructField(y,DoubleType,true), StructField(z,DoubleType,true))


In [2]:
// 정적 DataFrame의 dtatSchema 객체를 사용해 스티리밍의 schema 지정
val streaming = spark.readStream.schema(dataSchema)
    .option("maxFilesPerTrigger", 1).json("c:/SparkDG/data/activity-data/")

streaming: org.apache.spark.sql.DataFrame = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]


In [3]:
// groupBy Transformation 수행 (지연 연산) : 각 행동 별로 그룹을 짓고 count
val activityCounts = streaming.groupBy("gt").count()

activityCounts: org.apache.spark.sql.DataFrame = [gt: string, count: bigint]


In [4]:
// shuffle partition 수 줄이기
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [5]:
// 스트림 쿼리의 액션 정의. memory sink에 complete 출력 모드로 출력
val activityQuery = activityCounts.writeStream.queryName("activity_counts")
    .format("memory").outputMode("complete")
    .start()
// activityQuery.awaitTermination()

activityQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5a55442f


In [6]:
for (i <- 1 to 5) {
    spark.sql("select * from activity_counts").show()
    Thread.sleep(1000)             // 1초 마다 출력
}

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|49238|
|     stand|45539|
|stairsdown|37459|
|      walk|53024|
|  stairsup|41809|
|      null|41791|
|      bike|43187|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|73855|
|     stand|68309|
|stairsdown|56192|
|      walk|79536|
|  stairsup|62710|
|      null|62688|
|      bike|64781|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|86163|
|     stand|79694|
|stairsdown|65557|
|      walk|92792|
|  stairsup|73162|
|      null|73136|
|      bike|75579|
+----------+-----+

+----------+------+
|        gt| count|
+----------+------+
|       sit|110778|
|     stand|102464|
|stairsdown| 84286|
|      walk|119304|
|  stairsup| 94067|
|      null| 94033|
|      bike| 97175|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|123085|
|     stand|113849|
|stairsdown| 93648|
|      walk|132560|
|  stairs

In [7]:
spark.sql("select * from activity_counts").show()

+----------+------+
|        gt| count|
+----------+------+
|       sit|246159|
|     stand|227709|
|stairsdown|187264|
|      walk|265119|
|  stairsup|209082|
|      null|208954|
|      bike|215949|
+----------+------+



In [8]:
spark.sql("select * from activity_counts").show()

+----------+------+
|        gt| count|
+----------+------+
|       sit|258467|
|     stand|239093|
|stairsdown|196623|
|      walk|278375|
|  stairsup|219543|
|      null|219400|
|      bike|226746|
+----------+------+



In [9]:
spark.sql("select * from activity_counts").show()
// 파일 완전히 다 읽음

+----------+------+
|        gt| count|
+----------+------+
|       sit|258467|
|     stand|239093|
|stairsdown|196623|
|      walk|278375|
|  stairsup|219543|
|      null|219400|
|      bike|226746|
+----------+------+



In [10]:
// 행동, 모델로 그룹지은 후 가소도 센서를 의미하는 x,y,z 값의 평균 집계
val deviceModelStats = streaming.cube("gt", "model").avg()
   .drop("avg(Arrival_time)")
   .drop("avg(Creation_time)")
   .drop("avg(Index)")
   .writeStream.queryName("device_counts").format("memory").outputMode("complete")
   .start()

deviceModelStats: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@d48a0c5


In [13]:
spark.sql("select * from device_counts").show()

+----------+------+--------------------+--------------------+--------------------+
|        gt| model|              avg(x)|              avg(y)|              avg(z)|
+----------+------+--------------------+--------------------+--------------------+
|       sit|  null|-5.18435580910910...|2.377098624126100...|-2.00212909006953...|
|     stand|  null|-3.21765845070747...|3.048037742372213...|2.057814180385872E-4|
|       sit|nexus4|-5.18435580910910...|2.377098624126100...|-2.00212909006953...|
|     stand|nexus4|-3.21765845070747...|3.048037742372213...|2.057814180385872E-4|
|      null|  null|-0.00759977381558...|4.153276995856867...|0.003198222304879...|
|      null|  null| 7.16400359378949E-4|-0.00687358266758...|-0.00935555711419...|
|      walk|  null|-0.00441871154414...|0.002042781062758...|6.729268533703832E-5|
|      null|nexus4|-0.00759977381558...|4.153276995856867...|0.003198222304879...|
|      null|nexus4| 7.16400359378949E-4|-0.00687358266758...|-0.00935555711419...|
|   